In [148]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [150]:
#Hyper-parameters
num_epoch=10
batch_size=5
learning_rate=0.001
num_classes=10

In [152]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
training_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=False)

classes=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

Files already downloaded and verified
Files already downloaded and verified


In [153]:
#Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [174]:
#Defining CNN class
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(3,6, kernel_size=3, stride=1, padding=1)
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(6,16, kernel_size=3, stride=1, padding=1)
        # Fully connected layer
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        #print(x.size())
        x = x.view(-1,16*8*8)      # Flatten the tensor
        x = F.relu(self.fc1(x))          # Apply fully connected layer
        x = F.relu(self.fc2(x)) 
        x = self.fc3(x) 
        return x

In [176]:
#Model Defination
model = CNN().to(device)

In [178]:
#Defining loss and optimizer function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [180]:
n_total_step=len(train_loader)

for epoch in range(num_epoch):   
        
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            #labels=torch.tensor(labels)
            # Move data to device
            images = images.to(images)
            labels = labels.to(labels)

            # Forward pass
            output = model(images)
            loss = criterion(output, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()

            # Optimization step
            optimizer.step()
            
            if (i+1) % 10000 == 0:
                print(f"Epoch:[{epoch + 1}/{num_epoch}],  Loss: {loss.item():.4f}")

print("Training Finished.")


100%|██████████| 10000/10000 [06:22<00:00, 26.11it/s] 


Epoch:[1/10],  Loss: 2.2780


100%|██████████| 10000/10000 [05:20<00:00, 31.21it/s]s]       


Epoch:[3/10],  Loss: 1.3780


100%|██████████| 10000/10000 [10:23<00:00, 16.03it/s] 


...

Epoch:[8/10],  Loss: 1.2416


100%|██████████| 10000/10000 [02:06<00:00, 79.28it/s]


Epoch:[9/10],  Loss: 1.8681


100%|██████████| 10000/10000 [02:06<00:00, 79.35it/s]

Epoch:[10/10],  Loss: 1.0903
Training Finished.


In [182]:
#Checking accuracy of the model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode
    
    with torch.no_grad():  # Disable gradient calculation
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 24913/50000 with accuracy 49.83%
Checking accuracy on test data
Got 5020/10000 with accuracy 50.20%
